In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -q --upgrade pip
!pip install -q datasets
! pip install -q farm-haystack[colab,preprocessing,elasticsearch,metrics,inference,faiss-gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 64.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py

Creating K folds

In [3]:
import json

with open('/content/drive/MyDrive/DPR_train_v7.json', 'r', encoding='utf-8-sig') as f1:
  train_data = json.load(f1)

with open('/content/drive/MyDrive/DPR_dev_v7.json', 'r', encoding='utf-8-sig') as f2:
  dev_data = json.load(f2)

dict_data_list = train_data + dev_data


In [ ]:
broken_list = []

for each in dict_data_list:
  if len(each['hard_negative_ctxs']) != 5:
    broken_list.append(each)

len(broken_list)

138

In [4]:
import random

# Define the function to split the dataset into k folds
def split_into_k_folds(data, k):
    # Shuffle the data randomly
    random.shuffle(data)

    # Calculate the number of samples per fold
    fold_size = len(data) // k

    # Initialize an empty list to store the folds
    folds = []

    # Split the data into k folds
    for i in range(k):
        fold = data[i*fold_size : (i+1)*fold_size]
        folds.append(fold)

    return folds



In [5]:
dataset = dict_data_list

k = 5

# Split the dataset into k folds
folds = split_into_k_folds(dataset, k)

# Configure Document Store

In [7]:
from haystack import Pipeline
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import PreProcessor
from haystack.nodes import DensePassageRetriever
from haystack.schema import EvaluationResult, MultiLabel
from haystack.schema import Document
from bs4 import BeautifulSoup
from datasets import Dataset
import os
import re


doc_index = "quran_docs"
label_index = "quran_labels"


preprocessor = PreProcessor(
    split_by="word",
    language = 'fa',
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)

document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    return_embedding=True,
    embedding_dim=768,
)

with open('/content/drive/MyDrive/corpus/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

verses = soup.find_all("s")

dict_verse_list = []
for verse in verses:
  data_dict = {}
  context = re.sub("[\(\[].*?[\)\]]", "", verse.contents[0])
  context = re.sub(" +", " ", context)
  data_dict['id'] = verse.get('id')
  data_dict['text'] = context
  dict_verse_list.append(data_dict)

dataset = Dataset.from_list(dict_verse_list)

documents = []
for doc in dataset:
  documents.append(
      Document(
          content=doc["text"],
          meta={"title": '', "abstract": doc["text"], "pmid": doc["id"]},
      )
  )


docs_to_index = preprocessor.process(documents)

document_store.delete_documents()
document_store.write_documents(docs_to_index)

document_store.add_eval_data(filename="/content/drive/MyDrive/dev_SQuAD_format_v7.json", preprocessor=preprocessor)



Preprocessing: 100%|██████████| 6236/6236 [00:01<00:00, 4160.38docs/s]
Writing Documents: 10000it [00:20, 482.58it/s]
Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2707.75docs/s]
Writing Documents: 10000it [00:06, 1624.29it/s]


# cross validation

In [9]:
from haystack.document_stores import InMemoryDocumentStore
import itertools

for i in range(len(folds)):
  dev_set = folds[i]
  remains = [x for j,x in enumerate(folds) if j!=i]
  train_set = list(itertools.chain.from_iterable(remains))
  del remains

  with open('/content/train.json', 'w') as fp:
    json.dump(train_set, fp, ensure_ascii = False)

  with open('/content/dev.json', 'w') as fp:
    json.dump(dev_set, fp, ensure_ascii = False)

  train_filename = "train.json"
  dev_filename = "dev.json"

  query_model = "/content/drive/MyDrive/saved_models/dpr_parsbert/query_encoder"
  passage_model = "/content/drive/MyDrive/saved_models/dpr_parsbert/passage_encoder"

  save_dir = "/content/drive/MyDrive/saved_models/dpr_parsbert"

  retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=128,
    max_seq_len_passage=256,
  )
  retriever.train(
    # data_dir=doc_dir,
    data_dir = '/content',
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=2,
    grad_acc_steps=8,
    # evaluate_every=3000,
    evaluate_every=3000,
    save_dir=save_dir,
    embed_title=False,
    num_positives=1,
    num_hard_negatives=1,
  )

  dense_retriever = DensePassageRetriever.load(load_dir=save_dir, document_store=document_store)
  document_store.update_embeddings(retriever=dense_retriever)

  pipeline = Pipeline()
  pipeline.add_node(component=dense_retriever, name="DenseRetriever", inputs=["Query"])

  eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)
  eval_result = pipeline.eval(labels=eval_labels, params={"DenseRetriever": {"top_k": 1}})

  retriever_result = eval_result["DenseRetriever"]
  eval_result.save("/content/drive/MyDrive/dpr_results")

  saved_eval_result = EvaluationResult.load("/content/drive/MyDrive/dpr_results")
  metrics = saved_eval_result.calculate_metrics()
  print(f'Retriever - Recall (single relevant document): {metrics["DenseRetriever"]["recall_single_hit"]}')
  print(f'Retriever - Recall (multiple relevant documents): {metrics["DenseRetriever"]["recall_multi_hit"]}')
  print(f'Retriever - Mean Reciprocal Rank: {metrics["DenseRetriever"]["mrr"]}')
  print(f'Retriever - Precision: {metrics["DenseRetriever"]["precision"]}')
  print(f'Retriever - Mean Average Precision: {metrics["DenseRetriever"]["map"]}')
  print("*******************************************")

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Create embeddings: 100%|██████████| 6176/6176 [01:26<00:00, 84.87 Docs/s]
Documents Processed: 10000 docs [01:31, 108.84 docs/s]


Retriever - Recall (single relevant document): 0.35753575357535755
Retriever - Recall (multiple relevant documents): 0.35555555555555557
Retriever - Mean Reciprocal Rank: 0.35753575357535755
Retriever - Precision: 0.35753575357535755
Retriever - Mean Average Precision: 0.35555555555555557
*******************************************


Create embeddings: 100%|██████████| 6176/6176 [01:26<00:00, 84.67 Docs/s]
Documents Processed: 10000 docs [01:34, 106.37 docs/s]


Retriever - Recall (single relevant document): 0.35313531353135313
Retriever - Recall (multiple relevant documents): 0.3515851585158516
Retriever - Mean Reciprocal Rank: 0.35313531353135313
Retriever - Precision: 0.35313531353135313
Retriever - Mean Average Precision: 0.3515851585158516
*******************************************


Create embeddings: 100%|██████████| 6176/6176 [01:26<00:00, 85.17 Docs/s]
Documents Processed: 10000 docs [01:32, 108.06 docs/s]


Retriever - Recall (single relevant document): 0.37073707370737075
Retriever - Recall (multiple relevant documents): 0.36872853952061874
Retriever - Mean Reciprocal Rank: 0.37073707370737075
Retriever - Precision: 0.37073707370737075
Retriever - Mean Average Precision: 0.3686846945564121
*******************************************


Create embeddings: 100%|██████████| 6176/6176 [01:26<00:00, 85.59 Docs/s]
Documents Processed: 10000 docs [01:34, 105.30 docs/s]


Retriever - Recall (single relevant document): 0.35313531353135313
Retriever - Recall (multiple relevant documents): 0.35103510351035105
Retriever - Mean Reciprocal Rank: 0.35313531353135313
Retriever - Precision: 0.35313531353135313
Retriever - Mean Average Precision: 0.35103510351035105
*******************************************


Create embeddings: 100%|██████████| 6176/6176 [01:26<00:00, 84.84 Docs/s]
Documents Processed: 10000 docs [01:32, 108.24 docs/s]


Retriever - Recall (single relevant document): 0.37073707370737075
Retriever - Recall (multiple relevant documents): 0.3681326465979931
Retriever - Mean Reciprocal Rank: 0.37073707370737075
Retriever - Precision: 0.37073707370737075
Retriever - Mean Average Precision: 0.3681326465979931
*******************************************


In [ ]:
i= 2

a = [1, 5, 8, 9]

# b = a.pop(i)

b = [x for i,x in enumerate(a) if i!=3]

b

[1, 5, 8]

In [ ]:
from haystack import Pipeline
from haystack.document_stores import FAISSDocumentStore
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PreProcessor
from haystack.nodes import DensePassageRetriever
from haystack.schema import EvaluationResult, MultiLabel
from bs4 import BeautifulSoup
from datasets import Dataset
from haystack.schema import Document
import os
import re


doc_index = "quran_docs"
label_index = "quran_labels"


preprocessor = PreProcessor(
    split_by="word",
    language = 'fa',
    split_length=200,
    split_overlap=0,
    split_respect_sentence_boundary=False,
    clean_empty_lines=False,
    clean_whitespace=False,
)

document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    return_embedding=True,
    embedding_dim=768,
)

# document_store = InMemoryDocumentStore()

# document_store.delete_documents(index=doc_index)
# document_store.delete_documents(index=label_index)

# document_store.add_eval_data(
#     filename="/content/drive/MyDrive/dev_SQuAD_format_v7.json",
#     doc_index=doc_index,
#     label_index=label_index,
#     preprocessor=preprocessor,
# )

with open('/content/drive/MyDrive/corpus/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()

soup= BeautifulSoup(content, 'xml')

verses = soup.find_all("s")

dict_verse_list = []
for verse in verses:
  data_dict = {}
  context = re.sub("[\(\[].*?[\)\]]", "", verse.contents[0])
  context = re.sub(" +", " ", context)
  data_dict['id'] = verse.get('id')
  data_dict['text'] = context
  dict_verse_list.append(data_dict)

dataset = Dataset.from_list(dict_verse_list)

from haystack.schema import Document

documents = []
for doc in dataset:
  documents.append(
      Document(
          content=doc["text"],
          meta={"title": '', "abstract": doc["text"], "pmid": doc["id"]},
      )
  )

docs_to_index = preprocessor.process(documents)

document_store.delete_documents()
document_store.write_documents(docs_to_index)

# The add_eval_data() method converts the given dataset in json format into Haystack document and label objects. Those objects are then indexed in their respective document and label index in the document store. The method can be used with any dataset in SQuAD format.
document_store.add_eval_data(filename="/content/drive/MyDrive/dev_SQuAD_format_v7.json", preprocessor=preprocessor)



# Default label index: "label"

# document_store.write_documents(dev_data)

save_dir = '/content/drive/MyDrive/saved_models/dpr_parsbert'
dense_retriever = DensePassageRetriever.load(load_dir=save_dir, document_store=document_store)
document_store.update_embeddings(retriever=dense_retriever)
# document_store.optimize()

pipeline = Pipeline()

pipeline.add_node(component=dense_retriever, name="DenseRetriever", inputs=["Query"])


# query_pipeline.run(query = "What is Haystack?")
eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)

eval_result = pipeline.eval(labels=eval_labels, params={"DenseRetriever": {"top_k": 1}})

retriever_result = eval_result["DenseRetriever"]
# retriever_result.head()

eval_result.save("../")

saved_eval_result = EvaluationResult.load("../")
metrics = saved_eval_result.calculate_metrics()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 6236/6236 [00:00<00:00, 7930.77docs/s]
Writing Documents: 10000it [00:18, 528.05it/s]
Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 2697.30docs/s]
Writing Documents: 10000it [00:04, 2271.73it/s]
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Create embeddings: 100%|██████████| 6176/6176 [01:17<00:00, 92.96 Docs/s]
Documents Processed: 10000 docs [01:22, 121.03 docs/s]


In [ ]:
print(f'Retriever - Recall (single relevant document): {metrics["DenseRetriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["DenseRetriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["DenseRetriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["DenseRetriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["DenseRetriever"]["map"]}')

Retriever - Recall (single relevant document): 0.2662266226622662
Retriever - Recall (multiple relevant documents): 0.2628971230456379
Retriever - Mean Reciprocal Rank: 0.2662266226622662
Retriever - Precision: 0.2662266226622662
Retriever - Mean Average Precision: 0.2628071140447378


In [ ]:
document_store.get_label_count(index="label")  # returns: 92

1926

In [ ]:
document_store.get_document_count()

0

In [ ]:
eval_labels

[]

In [ ]:
import itertools
from haystack import Pipeline
from haystack.document_stores import FAISSDocumentStore

for i in range(len(folds)):
  dev_set = folds[i]
  remains = [x for j,x in enumerate(folds) if j!=i]
  train_set = list(itertools.chain.from_iterable(remains))
  del remains

  with open('/content/train.json', 'w') as fp:
    json.dump(train_set, fp, ensure_ascii = False)

  with open('/content/dev.json', 'w') as fp:
    json.dump(dev_set, fp, ensure_ascii = False)

  train_filename = "train.json"
  dev_filename = "dev.json"

  query_model = "/content/drive/MyDrive/saved_models/dpr_parsbert/query_encoder"
  passage_model = "/content/drive/MyDrive/saved_models/dpr_parsbert/passage_encoder"

  save_dir = "/content/drive/MyDrive/saved_models/dpr_parsbert"

  retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=128,
    max_seq_len_passage=256,
  )
  retriever.train(
    # data_dir=doc_dir,
    data_dir = '/content',
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=2,
    grad_acc_steps=8,
    # evaluate_every=3000,
    evaluate_every=3000,
    save_dir=save_dir,
    embed_title=False,
    num_positives=1,
    num_hard_negatives=1,
  )